## 파이토치 기초
- 데이터작업을 위한 두 개의 기초요소
- DataLoader: Dataset의 샘플과 레이블을 iterable하게 묶어줌

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

- 토치비전의 torchvision.datasets 모듈은 Dataset 객체를 가지고있다
- 샘플과 레이블을 수정하기 위한 transform and target_transform의 두가지 인자를 가지고 있다

In [3]:
#데이터셋 다운로드
training_data = datasets.FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data = datasets.FashionMNIST(
    root='data',
    train=False,
    download=True,
    transform=ToTensor(),
)

- DataLoader에 Dataset 객체를 인자로 전달한다.
- 우리의 데이터셋을 이터레이블하게 감쌀것이다 
- 자동배치와 셔플링,샘플링,멀티프로세서데이터로딩을 지원한다.
- 배치사이즈를 64로한다. => 데이터로더객체의 각 요소가 features와 labels의 64개 묶음으로 되어있다

In [4]:
batch_size = 64

#데이터 로더 생성
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"shape of X [N, C, H, W]: {X.shape}") 
    print(f"shape of y: {y.shape},{y.dtype}")
    break

shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
shape of y: torch.Size([64]),torch.int64


## 모델 만들기 
- 파이토치에서 신경망을 만들기위해서는 nn.Module에서 상속받은 class를 만든다
- 네트워크의 레이어를 __init__함수안에정의한다
- 데이터가 신경망을 어떻게 통과할지 forward함수에 특정한다

In [5]:
device =("cuda" if torch.cuda.is_available()
         else "cpu")
print(f"using {device} device")

using cpu device


In [6]:
# 모델 정의
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        #순차적으로 쌓는 레이어 정의
        self.linear_relu_stack = nn.Sequential(
            #완전연결 선형레이어와 활성화함수이다
            nn.Linear(28*28, 512),
            nn.ReLU(),
            # 512개의 입력을 받아 512개 출력
            nn.Linear(512, 512),
            # 비선형 변환
            nn.ReLU(),
            nn.Linear(512, 10),   
        )
    def forward(self, x):
        x = self.flatten(x)
        # 로짓은 클래스를 분류한 점수를 의미한다
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)


NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


## 모델 파라미터 최적화
- 모델 훈련을 위해 loss함수와 optimizer가 필요하다

In [7]:
# loss함수는 신경망의 마지막 레이어에서 출력된 점수를 확률로 바꿔준다
# 실제 레이블과 다를수록 손실이 증가한다
# softmax함수를 내부적으로 자동 적용한다
loss_fn = nn.CrossEntropyLoss()
# SGD 확률적 경사하강법이다
# 파라미터를 업데이트하여 loss함수의 손실을 최소화하는것이다
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

- 각 학습단계에서 모델은 (배치사이즈만큼) 제공되는 데이터셋에 대한 예측을 수행한다
- 예측 오류를 역전파하여 모델의 매개변수(가중치와 편향)을 조정한다

In [39]:
def train(dataloader, model, loss_fn, optimizer):
    # 6만개
    size = len(dataloader.dataset)
    model.train()
    # 938개의 배치
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        
        # 예측오류 계산
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # 역전파
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        if batch % 100 == 0:
            loss, current = loss.item(), (batch+1) * len(X)
            print(f"loss:{loss:>7f} [{current:>5d}/{size:>5d}]")

- 모델이 학습을 하는지 확인하기위해 테스트 데이터셋에대해 모델의 성능을 체크할것이다

In [74]:
def test(dataloader, model, loss_fn):
    #1만개
    size = len(dataloader.dataset)
    # 157개의 배치
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss:{test_loss:>8f} \n")

- 훈련은 여러번의 에포크에 걸쳐수행된다
- 모델은 더나은 예측을 위해 각 에포크마다 가중치를 학습한다
- 각에포크마다 모델의 정확도와 손실 볼수있다

In [75]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")


Epoch 1
-------------------------------
loss:2.275752 [64.000000/60000.000000]
loss:2.268226 [6464.000000/60000.000000]
loss:2.238841 [12864.000000/60000.000000]
loss:2.235321 [19264.000000/60000.000000]
loss:2.218423 [25664.000000/60000.000000]
loss:2.180218 [32064.000000/60000.000000]
loss:2.188128 [38464.000000/60000.000000]
loss:2.148860 [44864.000000/60000.000000]
loss:2.144933 [51264.000000/60000.000000]
loss:2.114131 [57664.000000/60000.000000]
Test Error: 
 Accuracy: 53.0%, Avg loss:2.107631 

Epoch 2
-------------------------------
loss:2.117478 [64.000000/60000.000000]
loss:2.107230 [6464.000000/60000.000000]
loss:2.036576 [12864.000000/60000.000000]
loss:2.051691 [19264.000000/60000.000000]
loss:1.996716 [25664.000000/60000.000000]
loss:1.931552 [32064.000000/60000.000000]
loss:1.953288 [38464.000000/60000.000000]
loss:1.868323 [44864.000000/60000.000000]
loss:1.873336 [51264.000000/60000.000000]
loss:1.809185 [57664.000000/60000.000000]
Test Error: 
 Accuracy: 59.6%, Avg lo

## 모델 저장
- 모델을 저장하는 일반 적인 방법은
- 파라미터들을 포함한 내부상태사전을 직렬화하는 것이다


In [76]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


## 모델 불러오기
- 모델을 불러올때는 모델 구조를 다시만들고 내부사전을 모델에 불러오는것이다

In [77]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [78]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
